In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from IPython.display import display, clear_output

from sklearn.ensemble import RandomForestClassifier
import joblib

#Load the Trained Model and the fit Scaler
rfModel = joblib.load('rfModel.joblib')
scaler = joblib.load('scaler.joblib')

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import ipywidgets as widgets
from IPython.display import display

In [3]:
# Set up your Spotify API credentials
client_id = '7f37bb631ed34cf29a487f88d5c5e32f'
client_secret = 'aff5f020a49b470f99d7a3c251f5b649'

# Authenticate with the Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Define a function to get track audio features
def get_track_audio_features(artist_name, track_name):
    # Search for the track
    results = sp.search(q=f"artist:{artist_name} track:{track_name}", type='track')

    # Check if the search results contain any tracks
    if len(results['tracks']['items']) == 0:
        print(f"No track found for '{artist_name} - {track_name}'")
        return None

    # Get the first track from the search results
    track = results['tracks']['items'][0]

    # Get the track's ID
    track_id = track['id']

    # Get audio features for the track
    audio_features = sp.audio_features(track_id)

    # Extract desired features
    track_features = {}
#    track_features['Artist'] = artist_name
#    track_features['Track'] = track_name
    track_features['Valence'] = audio_features[0]['valence']
    track_features['Acousticness'] = audio_features[0]['acousticness']
    track_features['Danceability'] = audio_features[0]['danceability']
    track_features['Duration (ms)'] = audio_features[0]['duration_ms']
    track_features['Energy'] = audio_features[0]['energy']
    track_features['Explicit'] = track['explicit']
    track_features['Instrumentalness'] = audio_features[0]['instrumentalness']
    track_features['Key'] = audio_features[0]['key']
    track_features['Liveness'] = audio_features[0]['liveness']
    track_features['Loudness'] = audio_features[0]['loudness']
    track_features['Mode'] = audio_features[0]['mode']
    track_features['Popularity'] = track['popularity']
    track_features['Speechiness'] = audio_features[0]['speechiness']
    track_features['Tempo'] = audio_features[0]['tempo']
    
    # Get the release date of the track
    release_date = track['album']['release_date']
    actual_year = release_date.split('-')[0]       # Extract the year from the release date

    return track_features, actual_year


In [4]:
def predict_decade(feature_array): 
    feature_array = np.array(feature_array).reshape(1, -1)
    spotify_track_scaled = scaler.transform(feature_array)  # Preprocess the sample data using the scaler

    # Make the prediction
    predicted_decade = rfModel.predict(spotify_track_scaled)
    
    return predicted_decade


In [5]:
def show_track_prediction(track_name, artist_name):
    # Get the user input
    #track_name = track_name_widget.value
    #artist_name = artist_name_widget.value

    # Print the user input
    print(f"Track Name: {track_name}")
    print(f"Artist Name: {artist_name}")

    track_features, actual_year = get_track_audio_features(artist_name, track_name)

    # Check if track features are available
    if track_features is not None:

        # Convert the dictionary to an array
        feature_array = [track_features[key] for key in track_features]

        predicted_decade = predict_decade(feature_array)
        
        return predicted_decade, actual_year


In [6]:
# Create text input widgets
track_name_widget = widgets.Text(description='Track Name:')
artist_name_widget = widgets.Text(description='Artist Name:')

# Create button widget
button = widgets.Button(description='Get Decade')
output = widgets.Output()

# Define function to handle button click event
def on_button_click(b):
    track_name = track_name_widget.value
    artist_name = artist_name_widget.value       
    
    with output:
        clear_output(wait=True) 

        predicted_decade, actual_year = show_track_prediction(track_name, artist_name)
        
        print(f"Predicted decade for '{track_name}' by {artist_name}: {predicted_decade[0]}")
        print(f"Actual year for '{track_name}' by {artist_name}: {actual_year}")

    
# Attach button click event handler
button.on_click(on_button_click)

#info = Markdown("""# SPOTIFY DECADE PREDICTOR""")
makeDisplay = widgets.VBox([track_name_widget, artist_name_widget,  button, output])
display(makeDisplay)